In [226]:
import pandas as pd
import numpy as np
import re
import MeCab
data = pd.read_excel('idiom_mod_data.xlsx')

In [227]:
# load raw dataset and drop nan
df = data[['어휘','수정']]
df = df.dropna(axis=0) # 애매한 의미 제거 
#df.head(30)

# Preprocessing

### 관용구 사전 생성
1. 관용구 데이터 정리
2. 다중 관용구 분리
3. 관용구 - 의미 쌍 데이터 생성

In [147]:
dics = df.values.tolist()

# ()처리 포함, 제거 두 예제로 나눔 
idiom_list_pre = []
meaning_list_pre = []
for d in dics:
    idiom = d[0]
    meaning = d[1]
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(parentheses) != 0:
        # 1. () 제외
        idiom_list_pre.append(\
                    re.sub('\(.*?\)', '', idiom))
        meaning_list_pre.append(meaning)

        # 2. ()안 조사 포함
        idiom_list_pre.append(\
                re.sub('[\(,\)]', '', idiom))
        meaning_list_pre.append(meaning)

    else:
        idiom_list_pre.append(idiom)
        meaning_list_pre.append(meaning)
print("idiom", len(idiom_list_pre))
print("meaning", len(meaning_list_pre))

5371
5371


In [148]:
# 다중 관용구 처리
idiom_list = []
meaning_list = []
for i, m in zip(idiom_list_pre, meaning_list_pre):
    # 의미 빈거 제거 (애매한 의미 등)
    idiom = i.replace('\n','')
    meaning = m.replace('\n','')
    #print(idiom)
    items = re.findall('\[.*?\]', idiom) #extracts string in bracket[] print items
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(items) != 0:
        # original
        idiom_list.append(\
                re.sub('\[.*?\]', '', idiom))
        meaning_list.append(meaning)
        
        # replace pair set
        rep_list = []
        str_list = idiom.split(' ')
        for word in str_list:
            for rep in items:
                if rep in word:
                    rep_list.append([word.replace(rep,''), rep])

        for pair in rep_list:
            # 유사어 여러개 있을때
            if '/' in pair[1]:
                reps = re.sub('[\[\]]','',pair[1]).split('/')
                #print(reps)
                for r in reps:
                    idiom_list.append(\
                            re.sub('\[.*?\]', '', (idiom.replace(pair[0]+pair[1], r))))
                    meaning_list.append(meaning)
            else:
                idiom_list.append(\
                        re.sub('\[.*?\]', '',\
                            (idiom.replace(pair[0]+pair[1], re.sub('[\[\]]','', pair[1])))))
                meaning_list.append(meaning)
    
    else:
        idiom_list.append(idiom)
        meaning_list.append(meaning)
        
print("idiom", len(idiom_list_pre))
print("meaning", len(meaning_list_pre))

5804
5804


# tokenizing with spacing token
1. 관용구 - 의미 쌍 토크나이징
2. 관용구는 띄어쓰기 토큰 없이 토크나이징 (띄어쓰기 토큰 포함시 제대로 토크나이징 안됨)
3. 의미는 띄어쓰기 토큰 포함하여 토크나이징 (문장에 포함시 띄어쓰기 정보 사용하기 위해)

In [166]:
# 형태소 분석, "/n, . 처리"
# save idiom dictionary to csv file
spacing_tkn = '__'

In [214]:
# 형태소 분석
# " 허리 가 __꼿꼿 하 ㄴ __철수"
mecab = MeCab.Tagger()

def tokenize(text, spacing_tkn=' '):
    # replace white space to spacing token
    text_spacing = text.replace(' ', spacing_tkn)
    text_spacing = text_spacing + "."
    # tokenize
    pos = mecab.parse(text_spacing).split('\n')
    tokens = []
    for i, p in enumerate(pos):
        #print(p)
        token = p.split('\t')[0]
        if token == "EOS":
            break
        
        tag = p.split('\t')[1].split(',')[0]
        # 연결 어미, 종결 어미 제외
        if tag == "EF" or tag == "SF" or token == "EOS":
            break
        if tag == "EC" and len(pos) == i+3:
            if token != "다":
                print(text)
                print(p)
            break
        if "다" in token and len(pos) == i+3:
            token = token.replace('다','')

        features = p.split(',')
        if features[-1] != '*':
            # 복합 일때
            if '+' in features[-1]: 
                for f in features[-1].split('+'):
                    tokens.append(f.split('/')[0])
            # 단일
            else:
                tokens.append(token)
        else:
            tokens.append(token)
    return tokens


In [215]:
# # test
# #tokenized = tokenize("분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡다.")
# tokenized = tokenize("가늠 보다", spacing_tkn=' ')
# print(tokenized)
# print(' '.join(tokenized))

['가늠', '보다', '.']
가늠 보다 .


In [211]:
# tokenizing (idiom, meaning)
idiom_df = pd.DataFrame(columns=['idiom', 'meaning'])
for i, m in zip(idiom_list, meaning_list):
    idiom = ' '.join(tokenize(i, spacing_tkn=' '))
    meaning = ' '.join(tokenize(m))
    idiom_df = idiom_df.append(pd.DataFrame([[idiom, meaning]], columns=['idiom', 'meaning']), ignore_index=True)

print(len(idiom_df))
#idiom_df.head(10)

5804


In [228]:
idiom_df.head(10)

,idiom,meaning
0,가꾸로 박히,머리 를 땅 으로 향하 ᆫ 채 떨어지
1,가난 들,가난 하 게 되
2,가난 이 들,가난 하 게 되
3,가난 이 파고들,더욱더 가난 하 아 지
4,가늠 보다,형편 이나 시세 따위 를 살피
5,가늠 을 보,형편 이나 시세 따위 를 살피
6,가늠 잡,형편 이나 시세 따위 를 살피
7,가늠 을 잡,형편 이나 시세 따위 를 살피
8,가늠 이 가,짐작 이나 헤아리 ᄆ 이 이루어지 거나 미치
9,가닥 을 잡,"분위기 , 상황 , 생각 따위 를 이치 나 논리 에 따르 아 바로잡"


In [213]:
# save tokenized idiom, meaning to csv file
idiom_df.to_csv('tokenized_idiom.csv', index=False)

# Idiom to meaning test

In [232]:
def raw_tokenize(text, spacing_tkn=' '):
    # replace white space to spacing token
    text_spacing = text.replace(' ', spacing_tkn)
    # tokenize
    pos = mecab.parse(text_spacing).split('\n')
    tokens = []
    for i, p in enumerate(pos):
        #print(p)
        token = p.split('\t')[0]
        # 연결 어미, 종결 어미 제외
        if token == "EOS":
            break
        features = p.split(',')
        if features[-1] != '*':
            # 복합 일때
            if '+' in features[-1]: 
                for f in features[-1].split('+'):
                    tokens.append(f.split('/')[0])
            # 단일
            else:
                tokens.append(token)
        else:
            tokens.append(token)
    return tokens

def idiom2mean(raw, dic):
    c_list = raw.split(' ')
    print("spacing info", c_list)
    print('\n')

    print("raw", raw)
    tokenized = ' '.join(raw_tokenize(raw, spacing_tkn=' '))
    print("tokenized", tokenized)
    for key in dic.keys():
        #print(key)
        if key in tokenized:
            meaning = tokenized.replace(key, dic[key])
            print("replaced meaning", meaning)
            meaning = meaning.replace(' ','').replace('__',' ')
            #print(meaning)

    for c in c_list:
        if c in meaning:
            meaning = meaning.replace(c, (c+' '))
    #print("used spacing info", meaning)

In [234]:
dic = {"하 ᆯ 말 을 잊 ":"놀랍 거나 __ 어처구니없 는 __ 일 을 __ 당하 여 __ 기 가 __ 막히",
       "가늠 이 가":"짐작 이나 __ 헤아리 ᄆ 이 __ 이루어지 거나 __ 미치",
       "힘 을 돌리":"어떤 __ 일 을 __ 이루 려고 __ 거기 에 __ 노력 을 __ 들 이",
       "황천 가":"사람 이 __ 죽",
       "가락 이 나":"행동 이 __ 서로 __ 잘 __ 들어맞"
      }
#raw = "그는 그들이 왜 자기만 잡아 왔는지 도무지 가늠이 가지 않아 답답했다."
raw = "가락이 나는 친구들이랑 해야 폰파라치도 돈 벌 수 있어요."
#raw = "외부에 힘을 돌리기는 커녕 눈을 돌릴 겨를도 없을 거요."
#raw = "황천 가는 수가 있어, 쥐도 새도 모르게. 뒤에서 어느 놈이 푹 찌르면 끝이라고."
idiom2mean(raw, dic)

spacing info ['가락이', '나는', '친구들이랑', '해야', '폰파라치도', '돈', '벌', '수', '있어요.']


raw 가락이 나는 친구들이랑 해야 폰파라치도 돈 벌 수 있어요.
tokenized 가락 이 나 는 친구 들 이랑 하 아야 폰 파라 하 지 도 돈 벌 ᆯ 수 있 어요 .
replaced meaning 행동 이 __ 서로 __ 잘 __ 들어맞 는 친구 들 이랑 하 아야 폰 파라 하 지 도 돈 벌 ᆯ 수 있 어요 .
